In [25]:
import pandas as pd
import geojson

def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["longitude"],
                                                    X["latitude"])),
                            properties=dict(icao=X["icao"],
                                            flight=X["flight_num"],
                                            operator=X["operator"],
                                            purpose=X["purpose"],
                                            altitude=X["altitude"],
                                            aircraft_type=X["type"],
                                            aircraft_url=X["url"])))
    df.apply(insert_features, axis=1)
    filename = filename + '.geojson'
    with open(filename, 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [26]:
data = pd.read_csv('data/aircraft_db.csv')
data['url'] = "https://www.airliners.net/search?keywords=" + data['type'].str.replace(' ', '+')
data.head()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,icao,regid,mdl,type,operator,url
0,000334,00000000,0000,Edra Super Petrel LS,Private owner,https://www.airliners.net/search?keywords=Edra...
1,000d77,00000000,0000,WT-9 Dynamic,Private owner,https://www.airliners.net/search?keywords=WT-9...
2,000d82,00000000,0000,Aerospool WT9 Dynamic,Private owner,https://www.airliners.net/search?keywords=Aero...
3,001100,00000000,0000,Unknown / Various,Code used by several aircraft,https://www.airliners.net/search?keywords=Unkn...
4,001108,00000000,0000,Gulfstream 690D,Ejercito de Colombia,https://www.airliners.net/search?keywords=Gulf...


In [27]:
data_private = pd.read_csv('data/private_owners.csv')
print(len(data_private))
data_private['purpose'] = 'private'
data_charter = pd.read_csv('data/unscheduled_charter.csv')
data_charter['purpose'] = 'charter'
print(len(data_charter))
data_rescue = pd.read_csv('data/rescue.csv')
data_rescue['purpose'] = 'rescue'
print(len(data_rescue))
data_military = pd.read_csv('data/military.csv')
data_military['purpose'] = 'military'
print(len(data_military))
data_space = pd.read_csv('data/space.csv')
data_space['purpose'] = 'space'
print(len(data_space))
data_training = pd.read_csv('data/academy_and_clubs.csv')
data_training['purpose'] = 'training'
print(len(data_training))

data2 = pd.concat([data_training, data_space, data_military, data_rescue, data_charter, data_private])

len(data2)

101111
30944
981
34739
234
4618


172627

In [28]:
data3 = pd.merge(data2, data, how='inner', on='icao')

In [29]:
data3['time'] = pd.to_datetime(data3['time'], unit = 'ms')

data3 = data3.sort_values(by='time')

data3.head()

,icao,altitude,latitude,longitude,time,flight_num,purpose,regid,mdl,type,operator,url
57522,49d13d,38000,46.20,12.56,2016-09-18 00:00:01.589,0,charter,ok-tvl,b738,Boeing 737-8FN,Travel Service,https://www.airliners.net/search?keywords=Boei...
43567,40676b,36350,52.37,-3.37,2016-09-18 00:00:01.758,0,charter,g-powh,b752,Boeing 757-256,Titan Airways,https://www.airliners.net/search?keywords=Boei...
45835,406905,35975,52.49,16.47,2016-09-18 00:00:01.991,0,charter,g-powi,a320,Airbus A320-233,Titan Airways,https://www.airliners.net/search?keywords=Airb...
43566,40676b,36325,52.37,-3.37,2016-09-18 00:00:03.109,0,charter,g-powh,b752,Boeing 757-256,Titan Airways,https://www.airliners.net/search?keywords=Boei...
43564,40676b,36300,52.37,-3.37,2016-09-18 00:00:04.133,0,charter,g-powh,b752,Boeing 757-256,Titan Airways,https://www.airliners.net/search?keywords=Boei...


In [30]:
data2geojson(data3, 'full_flights')

### Get only start and finish points

In [31]:
first_last_df = pd.DataFrame(columns=data3.columns)

coordinates, icaos, flight_nums, icaos, aircraft_types, aircraft_urls, purposes = [], [], [], [], [], [], []

for icao, icao_df in data3.groupby('icao'):
    for flight_num, flight_num_df in icao_df.groupby('flight_num'):
        first_last_df = first_last_df.append(flight_num_df.iloc[0], ignore_index=True)
        first_last_df = first_last_df.append(flight_num_df.iloc[-1], ignore_index=True)

In [32]:
data2geojson(first_last_df, 'aircrafts')

### Draw line through all coordinates

In [66]:
from random import randint
colors = []

for i in range(27):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

In [67]:
operators, coordinates, icaos, flight_nums, icaos, aircraft_types, aircraft_urls, altitudes, strokeColors, purposes = [], [], [], [], [], [], [], [], [], []

for icao, icao_df in data3.groupby('icao'):
    
    for flight_num, flight_num_df in icao_df.groupby('flight_num'):
        
        indexes = []
        
        for index, row in flight_num_df.iterrows():
            indexes.append(index)
            
        for i in range(len(indexes)):
            try:
                coordinates.append([[data3['longitude'].loc[indexes[i]], data3['latitude'].loc[indexes[i]]], [data3['longitude'].loc[indexes[i+1]], data3['latitude'].loc[indexes[i+1]]]])
                icaos.append(icao)
                flight_nums.append(flight_num)
                aircraft_urls.append(data3['url'].loc[indexes[i]])
                aircraft_types.append(data3['type'].loc[indexes[i]])
                altitudes.append(data3['altitude'].loc[indexes[i]])
                purposes.append(data3['purpose'].loc[indexes[i]])
                strokeColors.append(colors[flight_num])
                operators.append(data3['operator'].loc[indexes[i]])
            except Exception as e:
                continue


In [68]:
print(len(purposes))
print(len(strokeColors))

170080
170080


In [69]:
def data3geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.LineString((X["coordinates"])),
                            properties=dict(icao=X["icao"],
                                            strokeColor=X["strokeColor"],
                                            flight=X["flight_num"],
                                            purpose=X["purpose"],
                                            operator=X["operator"],
                                            altitude=X["altitude"],
                                            aircraft_type=X["type"],
                                            aircraft_url=X["url"])))
    df.apply(insert_features, axis=1)
    filename = filename + '.geojson'
    with open(filename, 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)
            

In [70]:
path_df = pd.DataFrame({'icao': icaos, 'coordinates': coordinates, 'flight_num': flight_nums,'altitude': altitudes, 'url': aircraft_urls, 'type': aircraft_types, 'strokeColor': strokeColors, 'purpose':purposes, 'operator':operators})    
data3geojson(path_df, 'full_flight_paths')